In [24]:
import requests
import json
import time
import pandas as pd
import datetime
from datetime import datetime
import numpy as np


In [35]:
def read_output(dct_in,**kwargs):
    columns=kwargs.get('columns',[])
    dct_out={}
    if len(columns)>0:
        for c in columns:
            dct_out[c]=[]
    else:
        for k in dct_in[0].keys():
            dct_out[k]=[]

    for cnt in range(0, len(dct_in)):
        for k in dct_out.keys():
            if k in dct_in[cnt].keys():
                dct_out[k].append(dct_in[cnt][k])
            else:
                dct_out[k].append(np.nan)
    return pd.DataFrame.from_dict(dct_out)

In [4]:
def add_datecols(df, **kwargs):
    date_col=kwargs.get('date_col', 'time')
    df['datestr']=[datetime.fromtimestamp(df[date_col].iloc[i]/1000) for i in range(0, len(df))]
    df['year_month']=[str(df.datestr.iloc[i].year)+'-'+str(df.datestr.iloc[i].month) for i in range(0, len(df))]

In [15]:
base_url = 'https://siastats.info/dbs/'
#daily data
supply_url = base_url + 'coinsupplydb.json' #SC supply in billions for each day - splits out miner rewards 

#below not used for rewards calculation, see all urls: https://siastats.info/api
reward_url = base_url + 'blockrewarddb.json' #reward per block (per end of day, i.e. 30k after block 270000)
block_data_url = base_url + 'miningdb.json' # avg. hashrate, difficulty and blocktime (in m) per day
fees_url= base_url + 'txfeesdb.json'
inflation_url = base_url + 'inflationdb.json' 
storage_url = base_url + 'storage.json' #storage in TB
hosts_url = base_url + 'activehosts.json' 
prices_url = base_url + 'storagepricesdb.json' #Avg storage pricing a renter would pay, in SC and USD and siafunds fees as % of the allowance



In [38]:
df_supply=read_output(requests.get(supply_url).json())
df_hosts=read_output(requests.get(hosts_url).json())

df_rewards=read_output(requests.get(reward_url).json())
df_block_data = read_output(requests.get(block_data_url).json())
df_fees=read_output(requests.get(fees_url).json())
df_inflation=read_output(requests.get(inflation_url).json())
df_storage=read_output(requests.get(storage_url).json())
df_prices = read_output(requests.get(prices_url).json(), columns=['date', 'price','usd','sfperfees'])


In [94]:
add_datecols(df_supply)
df_supply

,time,coinsupply,miners,foundation,datestr,year_month
0,1433548800000,0.005700,0.005700,0.0000,2015-06-06 02:00:00,2015-6
1,1433635200000,0.328499,0.328499,0.0000,2015-06-07 02:00:00,2015-6
2,1433721600000,0.627796,0.627796,0.0000,2015-06-08 02:00:00,2015-6
3,1433808000000,0.809032,0.809032,0.0000,2015-06-09 02:00:00,2015-6
4,1433894400000,1.035871,1.035871,0.0000,2015-06-10 02:00:00,2015-6
...,...,...,...,...,...,...
4720,1841356800000,69.850635,56.842035,13.0086,2028-05-08 02:00:00,2028-5
4721,1841443200000,69.854955,56.846355,13.0086,2028-05-09 02:00:00,2028-5
4722,1841529600000,69.859275,56.850675,13.0086,2028-05-10 02:00:00,2028-5
4723,1841616000000,69.863595,56.854995,13.0086,2028-05-11 02:00:00,2028-5


In [103]:
df_supply['miner_rewards']=(df_supply['miners']-df_supply['miners'].shift(1))*1000000000

In [106]:
df_supply_monthly = df_supply.groupby('year_month').agg(
    sum_rewards=('miner_rewards','sum'),
    circ_suply=('coinsupply','last'),
    end_month=('datestr','max')
    ).reset_index()


In [107]:
df_supply_monthly.to_csv('sia_rewards_monhtly.csv')

In [114]:
#number hosts
add_datecols(df_hosts, date_col='date')
df_hosts

,date,hosts,datestr,year_month
0,1506384000000,630,2017-09-26 02:00:00,2017-9
1,1506470400000,632,2017-09-27 02:00:00,2017-9
2,1506556800000,634,2017-09-28 02:00:00,2017-9
3,1506643200000,642,2017-09-29 02:00:00,2017-9
4,1506729600000,647,2017-09-30 02:00:00,2017-9
...,...,...,...,...
2053,1686873600000,351,2023-06-16 02:00:00,2023-6
2054,1686960000000,346,2023-06-17 02:00:00,2023-6
2055,1687046400000,343,2023-06-18 02:00:00,2023-6
2056,1687132800000,342,2023-06-19 02:00:00,2023-6


In [115]:
hosts_grp=df_hosts.groupby('year_month').agg(
    num_hosts=('hosts','mean'),
    end_month=('datestr','max')
    ).reset_index()

In [116]:
hosts_grp.to_csv('sia_hosts.csv')

In [47]:
#storage data (total and utilization)
add_datecols(df_storage, date_col='date')
df_storage['utilization']=df_storage['used']/df_storage['total']
df_storage_monthly = df_storage.groupby('year_month').agg(
    storage_supply=('total','last'),
    storage_used=('used','last'),
    avg_utilization=('utilization','mean')
).reset_index()

In [14]:
df_storage_monthly.to_csv('sia_storage.csv')

In [39]:
add_datecols(df_prices, date_col='date')


In [46]:
df_prices_monthly = df_prices.groupby('year_month').agg(
    price = ('price','mean'),
    usd = ('usd','mean'),
    sfperfees = ('sfperfees','mean')
).reset_index()

In [54]:
df_prices_monthly.to_csv('storage_prices.csv')